# Importing Required Libraries and Data Loading

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb
from lightgbm import LGBMRegressor

import shap

from itertools import combinations

In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [3]:
data_path = "/home/max1024/Downloads/drw-crypto-market-prediction/"
os.listdir(data_path)

['train.parquet', 'test.parquet', 'sample_submission.csv']

In [4]:
train_df = pd.read_parquet(data_path + "train.parquet")
print(train_df.shape)
train_df.head()

(525886, 786)


,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,...,X682,X683,X684,X685,X686,X687,X688,X689,X690,X691,X692,X693,X694,X695,X696,X697,X698,X699,X700,X701,X702,X703,X704,X705,X706,X707,X708,X709,X710,X711,X712,X713,X714,X715,X716,X717,X718,X719,X720,X721,X722,X723,X724,X725,X726,X727,X728,X729,X730,X731,X732,X733,X734,X735,X736,X737,X738,X739,X740,X741,X742,X743,X744,X745,X746,X747,X748,X749,X750,X751,X752,X753,X754,X755,X756,X757,X758,X759,X760,X761,X762,X763,X764,X765,X766,X767,X768,X769,X770,X771,X772,X773,X774,X775,X776,X777,X778,X779,X780,label
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.181844,-0.637860,0.006652,0.136870,0.116698,0.050156,0.063408,0.118437,1.498981,-0.058392,0.475105,0.598075,0.717269,1.128574,1.321473,1.415325,1.198338,0.988161,1.001487,1.028461,1.122363,1.026122,0.821888,0.629524,0.279410,1.941451,1.714323,1.625849,1.557562,1.108448,0.765369,0.519689,1.507433,1.265539,1.423863,1.183299,1.105583,0.896114,0.539748,0.413994,0.411489,0.305979,0.152076,0.089802,-0.019202,-0.030281,-0.118492,-0.114031,-0.295961,-0.264734,-0.457889,-0.413996,-0.514046,-0.468843,-0.607662,-0.561580,-1.234773,0.333121,-0.007063,0.165783,-0.308286,-0.008903,-0.464776,0.202286,0.063792,-0.085257,-0.236098,-0.013533,-0.092738,0.161414,0.133479,-0.188983,-0.063581,-0.005062,0.131056,0.057804,0.281357,-0.224598,0.111829,0.088571,0.450739,-0.215422,0.712970,-0.089143,0.066537,0.336256,0.680172,-0.271825,0.745266,0.081439,-0.130787,0.300110,0.848205,-0.242722,0.651824,...,-0.556556,-0.551802,0.035119,0.015866,-0.825437,-1.500472,-1.647912,0.013243,-0.904214,-1.663142,-1.424312,0.018708,-0.828889,-1.541797,-1.183578,-2.001567,-0.752834,-1.418354,-0.981090,-1.673004,-0.862372,-1.644269,-0.861785,-1.507782,-0.898247,-1.716997,-0.837155,-1.516427,-0.797252,-1.524046,-0.722323,-1.377873,0.203983,0.213461,-0.228590,-0.370086,-0.487077,-0.501514,-0.465749,0.018462,-0.880880,-0.918125,-0.683486,-1.093301,-1.070831,-1.140667,-0.779814,-1.251252,-1.298343,-1.434314,-0.728011,-1.205011,-1.052436,-1.178920,-0.418121,-0.758694,-0.657025,-0.752068,-0.090221,-0.260809,0.091926,0.330285,0.001635,0.016772,-0.435567,-0.854828,-0.194054,-0.386767,0.674001,0.000617,-0.532319,0.177538,0.617716,-0.625626,0.893549,-0.704807,-0.730939,0.363400,0.931868,0.577068,0.082875,-0.407380,-0.407366,-0.298108,-0.108466,0.040206,-0.033984,-0.178848,-0.596781,-0.868048,0.333753,-0.009992,-0.695595,-0.444077,-0.191238,-0.184251,-0.471897,-0.625428,-0.553991,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.489497,-0.075619,0.431594,0.522400,0.475255,0.380645,0.383859,0.433238,1.364390,-0.362606,0.218235,0.379769,0.528161,0.968490,1.172063,1.272317,1.127601,0.471929,0.671031,0.776974,0.927885,0.892181,0.711133,0.533984,0.235119,1.385692,1.396133,1.395087,1.386407,0.997484,0.676780,0.446211,1.454194,1.220531,1.358848,1.129289,1.078222,0.875519,0.522966,0.400787,0.394824,0.293150,0.153768,0.092357,-0.023043,-0.033101,-0.121427,-0.115972,-0.288401,-0.257674,-0.454711,-0.410828,-0.510239,-0.465008,-0.598882,-0.552937,0.307872,0.274915,-0.007548,0.112740,0.324003,-0.008903,0.552012,0.190584,0.058796,-0.091183,0.082906,-0.013543,0.720383,0.157031,0.128542,-0.189590,0.138390,-0.005502,0.794745,0.057448,0.276344,-0.224811,0.240839,0.086688,0.947686,-0.213934,0.708834,-0.090276,0.133019,0.334158,1.106949,-0.270885,0.743074,0.080262,-0.086308,0.299114,1.225641,-0.242266,0.650836,...,-0.555829,-0.551129,0.035119,0.015866,1.563655,2.848343,1.961337,3.398196,0.525775,0.972734,0.735608,1.268313,0.180116,0.341174,0.359868,0.020711,-0.041547,-0.071694,0.122862,0.214126,-0.448429,-0.849544,-0.212738,-0.

# Pruning Features

### Removing constanct features

In [5]:
constant_cols = []
for col in tqdm(train_df.columns):
    if train_df[col].nunique() == 1:
        constant_cols.append(col)

100%|██████████| 786/786 [00:08<00:00, 98.21it/s] 


In [6]:
train_df = train_df.drop(columns=constant_cols)
print(train_df.shape)

(525886, 786)


### Removing duplicated features

In [7]:
train_df = train_df.loc[:, ~train_df.T.duplicated()]
train_df.shape

(525886, 758)

### Removing low variance features

In [8]:
low_var_cols = []
for col in tqdm(train_df.columns):
    top_freq = train_df[col].value_counts(normalize=True).max()
    if top_freq > 0.95:
        low_var_cols.append(col)

100%|██████████| 758/758 [00:20<00:00, 37.82it/s]


In [9]:
train_df = train_df.drop(columns=low_var_cols)
print(train_df.shape)

(525886, 758)


### Removing highly correlated features

In [10]:
corr_matrix = train_df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

In [11]:
high_corr_cols = [col for col in tqdm(upper.columns) if any(upper[col] > 0.9)]

100%|██████████| 758/758 [00:00<00:00, 13613.32it/s]


In [12]:
len(high_corr_cols)

475

In [13]:
train_df = train_df.drop(columns=high_corr_cols)
print(train_df.shape)
train_df.head()

(525886, 283)


,bid_qty,ask_qty,buy_qty,sell_qty,X1,X2,X9,X10,X11,X17,X18,X19,X22,X25,X26,X27,X30,X33,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X81,X82,X83,X85,X86,X99,X100,X101,X102,X103,X105,X106,X107,X108,X109,X123,X124,X127,X141,X142,X143,X144,X145,X147,X148,X149,X150,X165,X169,X183,X184,X185,X189,X192,X193,X196,X213,X214,X217,X221,X224,X228,X229,X230,X231,X234,X235,X236,X237,X238,X239,X240,X248,X249,X256,X257,X258,X264,X265,X266,X269,X272,X273,X274,X277,X280,X304,X305,X306,X307,...,X555,X557,X558,X559,X560,X561,X562,X567,X593,X594,X595,X596,X597,X598,X605,X606,X607,X614,X615,X616,X618,X624,X630,X632,X633,X634,X636,X637,X641,X645,X646,X648,X649,X653,X657,X658,X660,X661,X662,X664,X665,X669,X673,X674,X676,X677,X681,X685,X686,X688,X689,X690,X692,X693,X697,X701,X702,X704,X705,X709,X713,X714,X716,X717,X718,X720,X721,X725,X729,X730,X732,X733,X737,X741,X742,X743,X744,X745,X746,X747,X748,X749,X750,X751,X752,X753,X754,X755,X756,X757,X758,X759,X761,X763,X767,X770,X771,X772,X774,label
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,0.181844,-0.637860,1.498981,-0.058392,0.475105,1.198338,0.988161,1.001487,1.026122,0.279410,1.941451,1.714323,1.108448,1.507433,-1.234773,0.333121,-0.007063,0.165783,-0.308286,-0.008903,-0.464776,0.202286,0.063792,-0.085257,-0.236098,-0.013533,0.450739,-0.215422,0.712970,0.066537,0.336256,-0.767850,0.392290,-0.010001,-0.189089,-0.175211,-0.231856,0.229596,0.235278,-0.264855,-0.148501,0.238922,-0.199899,-0.673642,-0.962329,1.583340,-0.011567,-0.214483,-0.302644,-0.312448,0.992699,0.595743,-0.455164,0.215349,-1.357882,0.078997,0.514655,0.198961,0.442216,0.110541,-0.203565,-1.101962,-0.018713,-0.196523,-0.866327,-0.455749,-1.347080,-0.528613,-0.807978,-1.161746,-1.193340,-1.135738,-1.246654,-0.939409,-0.554839,-0.252369,-0.125572,0.069598,0.467452,0.433544,0.456439,0.567190,0.618682,0.464005,0.724489,0.816991,0.961870,-0.489171,0.101241,0.157872,-0.199593,0.709540,0.085032,1.150329,0.032516,0.007036,...,-0.617290,-0.006257,-0.899497,-0.783004,0.457651,-0.674474,-0.587816,-1.069084,-1.759658,-0.263704,0.102322,-1.643456,-0.532390,0.274873,0.095638,-1.104399,0.716908,0.082552,0.706748,0.508370,0.030402,-0.780926,0.391442,-0.120655,-0.174005,-0.055969,-0.072285,0.024476,-0.267650,-0.544890,-1.370352,-0.645027,-1.005817,-0.864059,-0.574530,-0.002652,0.002295,0.011823,-0.004695,0.0045,0.016142,0.01839,0.016134,-0.260499,0.016004,0.019073,0.020183,0.015866,-0.825437,-1.647912,0.013243,-0.904214,-1.424312,0.018708,-2.001567,-1.673004,-0.862372,-0.861785,-1.507782,-1.516427,-1.377873,0.203983,-0.228590,-0.370086,-0.487077,-0.465749,0.018462,-1.093301,-1.251252,-1.298343,-0.728011,-1.205011,-0.758694,-0.260809,0.091926,0.330285,0.001635,0.016772,-0.435567,-0.854828,-0.194054,-0.386767,0.674001,0.000617,-0.532319,0.177538,0.617716,-0.625626,0.893549,-0.704807,-0.730939,0.363400,0.577068,-0.407380,0.040206,-0.596781,-0.868048,0.333753,-0.695595,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,0.489497,-0.075619,1.364390,-0.362606,0.218235,1.127601,0.471929,0.671031,0.892181,0.235119,1.385692,1.396133,0.997484,1.454194,0.307872,0.274915,-0.007548,0.112740,0.324003,-0.008903,0.552012,0.190584,0.058796,-0.091183,0.082906,-0.013543,0.947686,-0.213934,0.708834,0.133019,0.334158,0.878771,0.321132,-0.012274,-0.188981,-0.139300,0.849589,0.214880,0.218041,-0.264415,-0.136985,0.679914,-0.198764,-0.669527,0.717681,1.294016,-0.015420,-0.211817,-0.475121,0.765764,0.927812,0.554304,-0.443043,0.632736,-1.366976,0.142218,0.485653,0.153577,0.377093,0.446048,0.015531,-1.030538,0.251897,-0.034567,-0.823018,-0.086638,-1.117769,-0.066769,-0.369474,-0.693705,-0.769521,0.228490,-0.698824,-0.794767,-0.582503,-0.316399,-0.176520,0.032567,-0.185184,-0.768085,0.124892,-0.396644,-0.098488,0.093531,-2.067162,-1.090114,0.149263,-0.517539,-0.239279,-0.076416,-0.290893,0.655599,0.244035,0.931767,0.031524,0.016896,...,-0.622957,-0.006781,-0.901014,-0.783227,0.455007,-0.674931,-0.587967,-1.070057,-1.610331,-0.240773,0.093179,-1.596162,-0.515856,0.266103

### Removing useless features and selecting important ones

In [ ]:
def train(learning_rate=3e-4, num_iter=10000, early_stop=200):
    X = train_df.drop(columns=["label"])
    y = train_df["label"]
    
    n_splits = 5
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    params = {
        "learning_rate": learning_rate,
        "num_iterations": num_iter,
        "early_stopping_round": early_stop,
        "device": "gpu"
    }

    oof_preds = np.full(len(train_df), np.nan)

    shap_importance_list = []

    for i, (train_idx, val_idx) in enumerate(tscv.split(train_df)):
        print(f"Fold {i}:")
        print(f"  Train: index len={len(train_idx)}")
        print(f"  Test:  index len={len(val_idx)}")

        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = LGBMRegressor(**params)

        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])

        plt.figure()
        lgb.plot_metric(model)
        plt.show()

        oof_preds[val_idx] = model.predict(X_val)

        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X=X_val, y=y_val)
        shap_importance = np.abs(shap_values).mean(axis=0)
        shap_importance_list.append(shap_importance)

    mask = ~np.isnan(oof_preds)
    score = np.corrcoef(y[mask], oof_preds[mask])[0, 1]

    print(f"Pearson score: {score}")
        
    return np.mean(shap_importance_list, axis=0)

In [ ]:
shap_importance = train()

In [ ]:
cols = train_df.drop(["label"], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, shap_importance)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
imp_df

In [ ]:
plt.figure(figsize=(10, 15))
sns.barplot(x=imp_df['Importance'][:100], y=imp_df['Feature'][:100])

In [ ]:
plt.figure(figsize=(10, 15))
sns.barplot(x=imp_df['Importance'][:50], y=imp_df['Feature'][:50])

In [ ]:
unimportant_df = imp_df[imp_df['Importance'] == 0]
unimportant_cols = unimportant_df['Feature'].tolist()
len(unimportant_cols)

In [ ]:
train_df = train_df.drop(columns=unimportant_cols)
print(train_df.shape)
train_df.head()

In [ ]:
top_20_features_list = imp_df[:20]['Feature'].tolist()
print(len(top_20_features_list))
top_20_features_list

# Feature Engineering

### Row-wise feature engineering

In [ ]:
feature_list = train_df.drop(columns=["label"]).columns.tolist()
print(len(feature_list))
feature_list

In [ ]:
def row_wise_feat_engi(df):
    df = df.copy()
    new_features = {}

    new_features['row_mean'] = df[feature_list].mean(axis=1)
    new_features['row_std'] = df[feature_list].std(axis=1)
    new_features['row_max'] = df[feature_list].max(axis=1)
    new_features['row_min'] = df[feature_list].min(axis=1)
    new_features['row_sum'] = df[feature_list].sum(axis=1)

    for i in tqdm(range(19)):
        nth = round(0.05 + i * 0.05, 2)
        new_features['row_{}_quantile'.format(nth)] = df[feature_list].quantile(q=nth, axis=1)

    new_feats_df = pd.DataFrame(new_features, index=df.index)
    result_df = pd.concat([df, new_feats_df], axis=1)

    return result_df.copy()

In [ ]:
train_df = row_wise_feat_engi(train_df)
print(train_df.shape)
train_df.head()

### Nonlinear transformations

In [ ]:
def nonlinear_feat_engi(df):
    df = df.copy()
    new_features = {}

    for f in tqdm(top_20_features_list):
        new_features["{}_percentile_rank".format(f)] = df[f].rank(pct=True)
        new_features["{}_square".format(f)] = df[f].apply(lambda x: x**2)
        new_features["{}_cube".format(f)] = df[f].apply(lambda x: x**3)
        new_features["{}_sqrt".format(f)] = df[f].apply(lambda x: np.sqrt(np.abs(x)))
        new_features["{}_log1p".format(f)] = df[f].apply(lambda x: np.log1p(np.abs(x)))
        new_features["{}_exp".format(f)] = df[f].apply(lambda x: np.exp(x))
        new_features["{}_sin".format(f)] = df[f].apply(lambda x: np.sin(x))
        new_features["{}_cos".format(f)] = df[f].apply(lambda x: np.cos(x))
        new_features["{}_tanh".format(f)] = df[f].apply(lambda x: np.tanh(x))
        new_features["{}_abs".format(f)] = df[f].apply(lambda x: np.abs(x))
    
    new_feats_df = pd.DataFrame(new_features, index=df.index)
    result_df = pd.concat([df, new_feats_df], axis=1)

    return result_df.copy()

In [ ]:
train_df = nonlinear_feat_engi(train_df)
print(train_df.shape)
train_df.head()

### Interaction feature engineering

In [ ]:
def interaction_feat_engi(df):
    df = df.copy()
    new_features = {}

    for f1, f2 in combinations(top_20_features_list, 2):
        new_features[f'{f1}_{f2}_prod'] = df[f1] * df[f2]
        new_features[f'{f1}_{f2}_sum'] = df[f1] + df[f2]
        new_features[f'{f1}_{f2}_diff'] = df[f1] - df[f2]
        new_features[f'{f1}_{f2}_ratio'] = df[f1] / (df[f2] + 1e-5)
        new_features[f'{f1}_{f2}_max'] = df[[f1, f2]].max(axis=1)
        new_features[f'{f1}_{f2}_min'] = df[[f1, f2]].min(axis=1)
        new_features[f'{f1}_{f2}_absdiff'] = np.abs(df[f1] - df[f2])

    new_feats_df = pd.DataFrame(new_features, index=df.index)
    result_df = pd.concat([df, new_feats_df], axis=1)

    return result_df.copy()

In [ ]:
train_df = interaction_feat_engi(train_df)
print(train_df.shape)
train_df.head()

# Feature selection (again)

In [ ]:
shap_importance = train(num_iter=20000, early_stop=500)

In [ ]:
cols = train_df.drop(["label"], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, shap_importance)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
imp_df

In [ ]:
plt.figure(figsize=(10, 15))
sns.barplot(x=imp_df['Importance'][:100], y=imp_df['Feature'][:100])

In [ ]:
unimportant_df = imp_df[imp_df['Importance'] < 1e-4]
unimportant_cols = unimportant_df['Feature'].tolist()
len(unimportant_cols)

In [ ]:
train_df = train_df.drop(columns=unimportant_cols)
print(train_df.shape)
train_df.head()

In [ ]:
shap_importance2 = train(num_iter=20000, early_stop=500)

In [ ]:
cols = train_df.drop(["label"], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, shap_importance2)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
imp_df

In [ ]:
plt.figure(figsize=(10, 15))
sns.barplot(x=imp_df['Importance'][:100], y=imp_df['Feature'][:100])

In [ ]:
imp_df.tail(50)

In [ ]:
imp_df[:150]

In [ ]:
imp_df[:200].to_csv("../data/feature_importance_20250723_2205.csv", index=False)